# get locations from newspaper articles using Stanford CoreNLP

This notebook fetches RSS/Atom feeds and attempts to find the locations mentioned in the linked atricles. 

This implementation is just an example. It is inefficient and completely skips any error handling.

## Prequisites

* Download http://nlp.stanford.edu/software/stanford-corenlp-full-2018-10-05.zip and unzip it into the directory of this notebook
* Download http://nlp.stanford.edu/software/stanford-german-corenlp-2018-10-05-models.jar into the same directory and unzip it with `jar xf *models.jar`.
* install the python dependencies using `pip install -r requirements.txt`

In [1]:
import nltk

nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/nico/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## Create the tagger

In [2]:
# http://www.nltk.org/api/nltk.tag.html#module-nltk.tag.stanford

import os.path

from nltk.tag import StanfordNERTagger
from nltk import word_tokenize

StanfordNERTagger._JAR = 'stanford-corenlp-3.9.2.jar'

st = StanfordNERTagger('edu/stanford/nlp/models/ner/german.conll.germeval2014.hgc_175m_600.crf.ser.gz',
                       path_to_jar=os.path.join(os.getcwd(), 'stanford-corenlp-full-2018-10-05/', StanfordNERTagger._JAR))

def get_tags(text):
    return st.tag(word_tokenize(text))

def filter_locations(tags):
    return set([tag for tag, ttype in tags if ttype == 'LOCATION'])

## Fetch the recent newspaper articles and get their locations

In [3]:
import feedparser
import requests
import pyquery
from readability import Document
from IPython.display import display, Markdown

import warnings
warnings.filterwarnings("ignore") # future-warnings from readability

def url_to_text(u):
    resp = requests.get(u)
    d = pyquery.PyQuery(resp.text)
    doc = Document(str(d('article') or d('.content') or d))
    return pyquery.PyQuery(doc.content()).text().replace('\\n', ' ')

def render(title, link, tags):
    parts = [
        "[%s](%s)" % (title, link),
        ""
    ]
    for t in tags:
        parts.append("* %s" % t)
    display(Markdown("\n".join(parts)))


for feed_url in [
            'https://www.dlr.de/dlr/de/contentXXL/services/newsexport/rss.asmx/GetRSSNewsByTabId?tabid=10081',
            'https://www.nabu.de/rssfeed.php',
            'http://rss.sueddeutsche.de/rss/Topthemen']:
    
    f = feedparser.parse(feed_url)
    for e in f['entries']:
        resp = requests.get(e['link'])
        doc = Document(resp.text)
        content = url_to_text(e['link'])
        render(e['title'], e['link'], filter_locations(get_tags(content)))

[Neuer Staatssekretär des BMWi besucht Energieforschung im DLR Köln](https://www.dlr.de/dlr/de/desktopdefault.aspx/tabid-10081/151_read-33513/)

* TESIS
* Köln

[Testfeld Niedersachsen - Mobilitätsforschung mit Realdaten von der A39](https://www.dlr.de/dlr/de/desktopdefault.aspx/tabid-10081/151_read-33501/)

* Niedersachsen
* Cremlingen
* A2
* Braunschweig
* Wolfsburg

[Algen auf außerirdischer Mission: Atemluft für die ISS-Astronauten](https://www.dlr.de/dlr/de/desktopdefault.aspx/tabid-10081/151_read-33413/)

* Erde
* Deutschland
* ISS
* Mond
* Mars
* Friedrichshafen

[DLR führt mit Hilfe der Deutschen Lufthansa Flugversuche in Braunschweig durch](https://www.dlr.de/dlr/de/desktopdefault.aspx/tabid-10081/151_read-33460/)

* Braunschweig
* Braunschweig-Wolfsburg

[DLR auf dem Space Symposium 2019](https://www.dlr.de/dlr/de/desktopdefault.aspx/tabid-10081/151_read-33425/)

* Springs
* Jena
* Mond
* USA
* Thüringen
* Washington
* Colorado

...

[VroniPlag Wiki: Plagiatsjäger beanstanden mehr als 70 Seiten von Giffeys Dissertation](https://www.sueddeutsche.de/politik/giffey-dissertation-plagiate-vroniplag-1.4431726)

* Giffey
* Nordrhein-Westfalen

[Bundesliga: Gleich drei Neue für Schalke](https://www.sueddeutsche.de/sport/schalke-wagner-trainer-1.4431476)

* Dortmund
* Schalkes
* Hoffenheim
* Kerkrades
* Gelsenkirchen
* England
* Schalke
* Huddersfield
* China
* Ernst-Kuzorra-Weg
* Leipzig
* Paderborn
* Gladbach

[Europawahl: Was die Parteien wollen - und was sie ablehnen](https://www.sueddeutsche.de/politik/europawahl-wahlprogramme-parteien-1.4427514)

* Brüssel
* London
* Röszke
* Bitburg
* Peking
* FrankfurtMain
* Deutschland
* Brandenburg
* Köln
* China
* Trier
* Europa
* Straßburg

[Hauptversammlung: Die Enkel haben keine Lust auf RWE](https://www.sueddeutsche.de/wirtschaft/rwe-kohle-klimawandel-hauptversammlung-proteste-1.4431005)

* Dortmund
* Europas
* Deutschland
* Hambach
* Niederrhein
* Köln
* Innogy
* Europa

[CSU-Reformpläne:  "Söder will den Reichen ein Milliardengeschenk machen"](https://www.sueddeutsche.de/politik/markus-soeder-steuerreform-reaktionen-1.4430860)

* Österreichs
* Deutschland
* Washington
* New
* York
* Wien
* Ungarn
* Bulgarien
* Bayerns
* Jerusalem
* Bonn
* Berlin
* München